In [1]:
import matplotlib.pyplot as plt
import numpy as np
from custom_sepsis import *

/Users/luisastue/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import json
import numpy as np

# Define parameter sets
param_sets = {
    "sample_efficiency": {
        "buffer_size": 50000,
        "batch_size": 16,
        "train_freq": 1,
        "gradient_steps": 1,
        "exploration_fraction": 0.2,
        "exploration_final_eps": 0.05,
        "target_update_interval": 1000,
    },
    "asymptotic_rewards": {
        "buffer_size": 1000000,
        "batch_size": 256,
        "train_freq": 8,
        "gradient_steps": 20,
        "exploration_fraction": 0.05,
        "exploration_final_eps": 0.01,
        "target_update_interval": 1000,
    },
    "cumulative_rewards": {
        "buffer_size": 500000,
        "batch_size": 128,
        "train_freq": 4,
        "gradient_steps": 10,
        "exploration_fraction": 0.2,
        "exploration_final_eps": 0.05,
        "target_update_interval": 500,
    },
}

# Configuration
n_runs = 30
n_steps = 200000

# Flattened tasks for Pool
metric = "cumulative_rewards"
tasks = [
    (metric, param_sets[metric], run, n_steps)
    for run in range(23,30)
]

for task in tasks:
    train_and_log_rewards(task)


In [3]:
sample_eff = [result for result in results if result["metric"] == "sample_efficiency"]
asympt = [result for result in results if result["metric"] == "asymptotic_rewards"]
cumul = [result for result in results if result["metric"] == "cumulative_rewards"]

sample_eff_rewards = np.mean([result["episode_rewards"] for result in sample_eff], axis=0)
asympt_rewards = np.mean([result["episode_rewards"] for result in asympt], axis=0)
cumul_rewards = np.mean([result["episode_rewards"] for result in cumul], axis=0)

plt.plot(sample_eff_rewards, label="Sample Efficiency")
plt.plot(asympt_rewards, label="Asymptotic Rewards")
plt.plot(cumul_rewards, label="Cumulative Rewards")


NameError: name 'results' is not defined

In [ ]:

# Prepare the data for JSON
sample_eff_data = {
    "name": "Sample Efficiency",
    "info": {
        "description": "Mean rewards across 30 runs over 200000 steps for sample efficiency.",
        "n_steps": n_steps,
        "n_runs": n_runs,
        "params": param_sets["sample_efficiency"]
    },
    "mean_rewards": sample_eff_rewards.tolist(), # Convert numpy array to list
    "individual_runs": [result["episode_rewards"] for result in sample_eff] # include all runs for further analysis
}

asympt_data = {
    "name": "Asymptotic Rewards",
    "info": {
        "description": "Mean rewards across 30 runs over 200000 steps for asymptotic rewards.",
        "n_steps": n_steps,
        "n_runs": n_runs,
        "params": param_sets["asymptotic_rewards"]
    },
    "mean_rewards": asympt_rewards.tolist(), # Convert numpy array to list
    "individual_runs": [result["episode_rewards"] for result in asympt] # include all runs for further analysis
}

cumul_data = {
    "name": "Cumulative Rewards",
    "info": {
        "description": "Mean rewards across 30 runs over 200000 steps for cumulative rewards.",
        "n_steps": n_steps,
        "n_runs": n_runs,
        "params": param_sets["cumulative_rewards"]
    },
    "mean_rewards": cumul_rewards.tolist(), # Convert numpy array to list
    "individual_runs": [result["episode_rewards"] for result in cumul] # include all runs for further analysis
}

# Save the data to JSON files
with open("json/dqn/sample_eff_results.json", "w") as f:
    json.dump(sample_eff_data, f, indent=4)

with open("json/dqn/asympt_results.json", "w") as f:
    json.dump(asympt_data, f, indent=4)

with open("json/dqn/cumul_results.json", "w") as f:
    json.dump(cumul_data, f, indent=4)